# "Ranking learning"

- toc: false
- comments: true
- hide: true
- search_exclude: true

So we have some kind of reading club at our company where we normally share papers that we find interesting, so one of us has to explain the paper to the rest. 
This is week was my turn and since we had lately been talking about recommendation I thought on looking for a paper on Ranking. But instead of finding a SOA paper,
I thought it might be interesting to go into the roots and start simple since I hadnt touched much about this topic before. I finally decided to talk about the 
paper by Microsoft named "" wchich is not precisely new. Once I started reading it some references to previous work pop out as expected but there was a common term in 
all of them, a more basic concept named Ordinal Regression. So I decided to read about this and explain this concept before commenting on the paper. So I am gonna 
recreate my talk here starting from Ordinal Regression and touching later on Ranking as explained in the Microsoft paper. 

So to clarify, Ordinal regression is actually the same as Ranking Learning only that the latter is used in the machine learning field. The authors in the paper mention Ordinal Regression itself as a model and I think what they really refer to is the generalized linear model that I will present next (actually Wikipedia will).

Wikipedia has a nice entry on this topic so I am just going to copy the relevant parts and comment them when needed:

## Linear models for ordinal regression
Ordinal regression can be performed using a generalized linear model (GLM) that fits both a coefficient vector and a set of thresholds to a dataset. Suppose one has a set of observations, represented by length-p vectors $\small x_1$ through $\small x_n$, with associated responses $\small y_1$ through $\small y_n$, where each $\small y_i$ is an ordinal variable on a scale $\small 1, ..., K$. For simplicity, and without loss of generality, we assume $\small y$ is a non-decreasing vector, that is, $\small y_i \leq y_{i+1}$. To this data, one fits a length-p coefficient vector $\small \bf w$ and a set of thresholds $\small \theta_1, ..., \theta_{K−1}$ with the property that $\small θ1 < \theta_2 < ... < \theta_{K−1}$. This set of thresholds divides the real number line into $\small K$ disjoint segments, corresponding to the $\small K$ response levels.
The model can now be formulated as 
$$
Pr(y\leq i|\bf{x}) = \sigma(\theta_i - \bf{w \cdot x})
$$

There are different options for the $\small\sigma$ function (inverse link funciton in GLM language) which is our cummulative distribution function, and I am going to mention two:
- logistic function
- probit function

When we use the logistic function we assume that our error term follows a logistic distribution with $\small \mu = 0$ and $s = 0$ (scale parameter) conditioned on $\small x$ and when we use the probit function we assume our error term follows a normal distribution with $\small \mu = 0$ and $\small \sigma = 0$ conditioned on $\small x$. Both distributions have a similar shape but the logistic distribution has heavier tails, hence being more robust.

In [2]:
import numpy as np
from scipy.stats import norm, logistic
import matplotlib.pyplot as plt

fig, ax = plt.figure(1, 2, figsize=(15, 5))

# generate points to plot logistic distribution
x_logistic = np.linspace(logistic.ppf(0.01), logistic.ppf(0.99), 100)
y_logistic = logistic.pdf(x_logistic)

# generate points to plot normal distribution
x_norm = np.linspace(norm.ppf(0.01), norm.ppf(0.99), 100)
y_norm = norm.pdf(x_norm)



ax[0].plot


So maybe you are still wondering where this error term appears or how do we even fit this model, so the wikipedia article goes on and talks about a latent variable model. 

## Latent variable model
The probit version of the above model can be justified by assuming the existence of a real-valued latent variable (unobserved quantity) $\small y*$, determined by
$$
y* = \bf{w\cdot x} + \epsilon
$$
where the error term is normally distributed conditioned on $\small x$ as mentioned above. The response variable $\small y$ results from an "incomplete measurement" of $\small y*$, where one only determines the interval into which $\small y*$ falls:

$$
y =
    \begin{cases}
      1 & \text{if $y* \leq \theta_1$}\\
      2 & \text{if $\theta_1 \leq y* \leq \theta_2$}\\
      \vdots\\
      K & \text{if $\theta_{k-1} \leq y*$}
    \end{cases}  
$$